In [1]:
from common.alpha import Alpha
from common.td import TD

import datetime
import pandas as pd
import numpy as np

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

pd.set_option("display.max_columns", 100)
init_notebook_mode(connected=True)

In [2]:
alpha = Alpha()

td_client = TD()

In [3]:
end = datetime.datetime.now().timestamp()*1000
start = end - 2592000000
print(start, end)

1560054199280.4739 1562646199280.4739


In [4]:
# Goal: Pull minute data for the last 5 years for specific ETF stocks
data = td_client.get_price_history(symbol='DGAZ', period_type='day', period=10, frequency_type='minute', frequency=1, start_date=int(start), end_date=int(end), needExtendedHoursData=True)

In [5]:
# Converting dict to pandas df
df = pd.DataFrame.from_dict(data)
df.head()

,candles,symbol,empty
0,"{'open': 165.66, 'high': 165.66, 'low': 165.66...",DGAZ,False
1,"{'open': 165.61, 'high': 165.61, 'low': 165.61...",DGAZ,False
2,"{'open': 165.4, 'high': 165.4, 'low': 165.31, ...",DGAZ,False
3,"{'open': 165.35, 'high': 165.35, 'low': 165.0,...",DGAZ,False
4,"{'open': 164.55, 'high': 164.55, 'low': 164.18...",DGAZ,False


In [6]:
# Expanding 'candles' col to multiple columns
df = pd.concat([df.drop(['candles'], axis=1), df['candles'].apply(pd.Series)], axis=1)
df.head()

,symbol,empty,open,high,low,close,volume,datetime
0,DGAZ,False,165.66,165.66,165.66,165.66,100.0,1.560166e+12
1,DGAZ,False,165.61,165.61,165.61,165.61,100.0,1.560166e+12
2,DGAZ,False,165.40,165.40,165.31,165.31,200.0,1.560166e+12
3,DGAZ,False,165.35,165.35,165.00,165.00,668.0,1.560166e+12
4,DGAZ,False,164.55,164.55,164.18,164.18,350.0,1.560167e+12


In [7]:
# Create 'date' col from 'datetime'. Converting from epoch time from milliseconds to seconds, then to date
df['date'] = df['datetime'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0).strftime('%Y-%m-%d %H:%M'))
df = df.set_index('date')
df.head()

,symbol,empty,open,high,low,close,volume,datetime
date,,,,,,,,
2019-06-10 06:24,DGAZ,False,165.66,165.66,165.66,165.66,100.0,1.560166e+12
2019-06-10 06:26,DGAZ,False,165.61,165.61,165.61,165.61,100.0,1.560166e+12
2019-06-10 06:34,DGAZ,False,165.40,165.40,165.31,165.31,200.0,1.560166e+12
2019-06-10 06:35,DGAZ,False,165.35,165.35,165.00,165.00,668.0,1.560166e+12
2019-06-10 06:39,DGAZ,False,164.55,164.55,164.18,164.18,350.0,1.560167e+12


In [9]:
df.sort_index(inplace=True, ascending=False)
df.head()

,symbol,empty,open,high,low,close,volume,datetime
date,,,,,,,,
2019-07-08 18:37,DGAZ,False,138.54,138.54,138.54,138.54,100.0,1.562629e+12
2019-07-08 17:53,DGAZ,False,138.78,138.78,138.78,138.78,90.0,1.562626e+12
2019-07-08 17:50,DGAZ,False,138.54,138.54,138.54,138.54,424.0,1.562626e+12
2019-07-08 17:30,DGAZ,False,138.38,138.38,138.38,138.38,361.0,1.562625e+12
2019-07-08 17:23,DGAZ,False,138.82,138.82,138.82,138.82,100.0,1.562625e+12


In [ ]:
INCREASING_COLOR = 'g'
DECREASING_COLOR = 'r'

In [ ]:
data = [ dict(
    type = 'candlestick',
    open = df['open'],
    high = df['high'],
    low = df['low'],
    close = df['close'],
    x = df.index,
    yaxis = 'y2',
    name = 'Historic Data',
    increasing = dict( line = dict( color = INCREASING_COLOR ) ),
    decreasing = dict( line = dict( color = DECREASING_COLOR ) ),
) ]

layout=dict()

fig = dict( data=data, layout=layout )

In [ ]:
fig['layout'] = dict()
fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
fig['layout']['xaxis'] = dict( rangeselector = dict( visible = True ) )
fig['layout']['yaxis'] = dict( domain = [0, 0.2], showticklabels = False )
fig['layout']['yaxis2'] = dict( domain = [0.2, 0.8] )
fig['layout']['legend'] = dict( orientation = 'h', y=0.9, x=0.3, yanchor='bottom' )
fig['layout']['margin'] = dict( t=40, b=40, r=40, l=40 )

In [ ]:
rangeselector=dict(
    visibe = True,
    x = 0, y = 0.9,
    bgcolor = 'rgba(150, 200, 250, 0.4)',
    font = dict( size = 13 ),
    buttons=list([
        dict(count=1,
             label='reset',
             step='all'),
        dict(count=1,
             label='1 yr',
             step='year',
             stepmode='backward'),
        dict(count=3,
            label='1 month',
            step='month',
            stepmode='backward'),
        dict(count=1,
            label='1 day',
            step='day',
            stepmode='backward'),
        dict(step='all')
    ]))
    
fig['layout']['xaxis']['rangeselector'] = rangeselector

In [ ]:
# Add EMA lines
df['9_day_ema'] = df['close'].ewm(span=9, adjust=False).mean()
df['20_day_ema'] = df['close'].ewm(span=20, adjust=False).mean()


mv_y = df['9_day_ema']
mv_x = list(df.index)

fig['data'].append( dict( x=mv_x, y=mv_y, type='scatter', mode='lines', 
                         line = dict( width = 1 ),
                         marker = dict( color = '#2ca02c' ),
                         yaxis = 'y2', name='9 Day Exp Moving Average' ) )

mv_y = df['20_day_ema']
mv_x = list(df.index)

fig['data'].append( dict( x=mv_x, y=mv_y, type='scatter', mode='lines', 
                         line = dict( width = 1 ),
                         marker = dict( color = '#17becf' ),
                         yaxis = 'y2', name='20 Day Exp Moving Average' ) )

In [ ]:
colors = []

for i in range(len(df['close'])):
    if i != 0:
        if df['close'].iloc[i] > df['close'].iloc[i-1]:
            colors.append(INCREASING_COLOR)
        else:
            colors.append(DECREASING_COLOR)
    else:
        colors.append(DECREASING_COLOR)

In [ ]:
fig['data'].append( dict( x=df.index, y=df['volume'],                         
                         marker=dict( color=colors ),
                         type='bar', yaxis='y', name='Volume' ) )

In [ ]:
iplot(fig, filename='charts/dgaz_test2', validate=False)